In [3]:
import dgl.data

dataset = dgl.data.GINDataset('PROTEINS', self_loop=True)
print(f'Node feature dimensionality: {dataset.dim_nfeats}')
print(f'Number of graph categories: {dataset.gclasses}')

Extracting file to /root/.dgl/GINDataset


In [10]:
import torch
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
num_train = int(num_examples * 0.8)

train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(
    dataset, sampler=train_sampler, batch_size=5, drop_last=False
)
test_dataloader = GraphDataLoader(
    dataset, sampler=test_sampler, batch_size=5, drop_last=False
)

In [11]:
train_dataloader

In [18]:
it = iter(train_dataloader)
batch = next(it)
print(batch)

[Graph(num_nodes=260, num_edges=1158,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={}), tensor([1, 0, 0, 0, 0])]


## A Batched Graph in DGL

In [19]:
batched_graph ,labels = batch

In [20]:
batched_graph, labels

(Graph(num_nodes=260, num_edges=1158,
       ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
       edata_schemes={}),
 tensor([1, 0, 0, 0, 0]))

In [21]:
graphs = dgl.unbatch(batched_graph)
print(graphs)

[Graph(num_nodes=27, num_edges=127,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=136, num_edges=560,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=22, num_edges=104,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=50, num_edges=224,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=25, num_edges=143,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})]


## Modeling

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [23]:
from dgl.nn import GraphConv, GATConv, GINConv, APPNPConv

### GCN line

In [31]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
        
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        return dgl.mean_nodes(g, 'h')

### GINConv line

## Training

### original version ( not using GPU )

In [ ]:
GCN_model = GCN(dataset.dim_nfeats, 16, dataset.gclasses)
GCN_optimizer = torch.optim.Adam(GCN_model.parameters(), lr=0.01)

for epoch in tqdm(range(100)):
    for batched_graph, labels in train_dataloader:
        
        pred = GCN_model(batched_graph, batched_graph.ndata['attr'].float())
        loss = F.cross_entropy(pred, labels)
        GCN_optimizer.zero_grad()
        loss.backward()
        GCN_optimizer.step()
        
num_correct = 0
num_tests = 0
for batched_graph, labels in test_dataloader:
    pred = GCN_model(batched_graph, batched_graph.ndata['attr'].float())
    num_correct += (pred.argmax(1) == labels).sum().item()
    num_tests += len(labels)
    
print(f'Test accuracy: {num_correct / num_tests}')

### GPU version

In [78]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
GCN_model = GCN_model.to(device)
GCN_optimizer = torch.optim.Adam(GCN_model.parameters(), lr=0.01)

In [79]:
GCN_model.train()
for i in tqdm(range(10000)):
    loss_list = []
    true_samples = 0
    num_samples = 0
    for batch_id, batch_data in enumerate(train_dataloader):
        bg, labels = batch_data
        graph_feats = bg.ndata.pop('attr').float()
        bg = bg.to(device)
        graph_feats, labels = graph_feats.to(device), labels.to(device)
        logits = GCN_model(bg, graph_feats) ##
        loss = F.cross_entropy(logits, labels)
        true_samples += (logits.argmax(1) == labels.long()).float().sum().item()
        num_samples += len(labels)
        loss_list.append(loss.item())
        GCN_optimizer.zero_grad()
        loss.backward()
        GCN_optimizer.step()
    if i % 100 == 0:
        print(f"Epoch {i:05d} | Loss : {np.mean(loss_list):.4f} | Accuracy : {true_samples/num_samples:.4f}")

  0%|          | 1/10000 [00:01<2:48:12,  1.01s/it]

Epoch 00000 | Loss : 0.5793 | Accuracy : 0.7404


  1%|          | 101/10000 [01:32<2:32:00,  1.09it/s]

Epoch 00100 | Loss : 0.5071 | Accuracy : 0.7618


  2%|▏         | 201/10000 [03:03<2:28:29,  1.10it/s]

...

 98%|█████████▊| 9801/10000 [2:30:51<03:01,  1.10it/s]

Epoch 09800 | Loss : 0.4925 | Accuracy : 0.7708


 99%|█████████▉| 9901/10000 [2:32:22<01:27,  1.13it/s]

Epoch 09900 | Loss : 0.4912 | Accuracy : 0.7798


100%|██████████| 10000/10000 [2:33:54<00:00,  1.08it/s]


Epoch 100 

| Loss : 0.5351 | Accuracy : 0.7517

Epoch 10000


### Hyperparameter + Ray tune
Fail

In [68]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [71]:
config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

In [76]:
GCN_model = GCN(dataset.dim_nfeats, 16, dataset.gclasses)
GCN_optimizer = torch.optim.Adam(GCN_model.parameters(), lr=0.01)

In [65]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    if torch.cuda.device_count() > 1:
        GCN_model = nn.DataParallel(GCN_model)
GCN_model.to(device)

DataParallel(
  (module): DataParallel(
    (module): GCN(
      (conv1): GraphConv(in=3, out=16, normalization=both, activation=None)
      (conv2): GraphConv(in=16, out=2, normalization=both, activation=None)
    )
  )
)

In [ ]:
GCN_model.train()
for i in tqdm(range(10000)):
    loss_list = []
    true_samples = 0
    num_samples = 0
    for batch_id, batch_data in enumerate(train_dataloader):
        bg, labels = batch_data
        graph_feats = bg.ndata.pop('attr').float()
        bg = bg.to(device)
        graph_feats, labels = graph_feats.to(device), labels.to(device)
        logits = GCN_model(bg, graph_feats) ##
        loss = F.cross_entropy(logits, labels)
        true_samples += (logits.argmax(1) == labels.long()).float().sum().item()
        num_samples += len(labels)
        loss_list.append(loss.item())
        GCN_optimizer.zero_grad()
        loss.backward()
        GCN_optimizer.step()
    if i % 100 == 0:
        print(f"Epoch {i:05d} | Loss : {np.mean(loss_list):.4f} | Accuracy : {true_samples/num_samples:.4f}")